<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import plotly.express as px

import warnings

# Игнорирование предупреждения
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies).

In [ ]:
query_3_1 = f'''
                select *
                from vacancies
'''

In [ ]:
vacancies = pd.read_sql_query(query_3_1, con=connection)
vacancies.shape[0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers).

In [ ]:
query_3_2 = f'''
                select *
                from employers
'''

In [ ]:
employers = pd.read_sql_query(query_3_2, con=connection)
employers.shape[0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [ ]:
query_3_3 = f'''
                select *
                from areas
'''

In [ ]:
areas = pd.read_sql_query(query_3_3, con=connection)
areas.shape[0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [ ]:
query_3_4 = f'''
                select *
                from industries
'''

In [ ]:
industries = pd.read_sql_query(query_3_4, con=connection)
industries.shape[0]

294

***

In [ ]:
print(f'В среднем на одного работадателя приходится {round(vacancies.shape[0] / employers.shape[0], 2)} вакансии')

В среднем на одного работадателя приходится 2.09 вакансии


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [4]:
query_4_1 = f'''
                select count(v.id), a.name as area
                from vacancies v
                join areas a on v.area_id = a.id
                group by a.name
                order by 1
'''

In [5]:
df_4_1 = pd.read_sql_query(query_4_1, con=connection)
df_4_1

,count,area
0,1,Монино
1,1,Гудермес
2,1,Китай
3,1,Звенигород
4,1,Северо-Енисейский (Красноярский край)
...,...,...
764,1892,Алматы
765,2006,Новосибирск
766,2112,Минск
767,2851,Санкт-Петербург


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [6]:
query_4_2 = f'''
                select count(id)
                from vacancies
                where salary_from is not null or salary_to is not null
'''

In [7]:
pd.read_sql_query(query_4_2, con=connection)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [8]:
query_4_3 = f'''
                select round(avg(salary_from)) as from, round(avg(salary_to)) as to
                from vacancies
'''

In [9]:
df_4_3 = pd.read_sql_query(query_4_3, con=connection)
df_4_3

,from,to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [10]:
query_4_4 = f'''
                select count(id), schedule, employment
                from vacancies
                group by schedule, employment
                order by 1 desc
'''

In [11]:
pd.read_sql_query(query_4_4, con=connection)

,count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость
5,569,Полный день,Стажировка
6,367,Вахтовый метод,Полная занятость
7,347,Полный день,Частичная занятость
8,312,Гибкий график,Частичная занятость
9,141,Полный день,Проектная работа


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [12]:
query_4_5 = f'''
                select count(id), experience
                from vacancies
                group by experience
                order by 1
'''

In [13]:
pd.read_sql_query(query_4_5, con=connection)

,count,experience
0,1337,Более 6 лет
1,7197,Нет опыта
2,14511,От 3 до 6 лет
3,26152,От 1 года до 3 лет


***

In [14]:
mask = df_4_1['count'] >= 50

val = round(df_4_1[mask]['count'].sum() / df_4_1['count'].sum(), 3)

fig_4_1 = px.bar(
    df_4_1[mask],
    x='area',
    y='count',
    text='area',
    title=f'{val * 100}% количества всех вакансий',
    labels={'Вакансии': 'Количество вакансий', 'Город': 'Город'})

fig_4_1.update_traces(texttemplate='%{text}', textposition='outside')
fig_4_1.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_4_1.show()

In [15]:
df_4_3 = pd.read_sql_query(f'''
                              select round(coalesce((salary_to + salary_from) / 2, salary_to, salary_from)) as salary
                              from vacancies
                              where salary_from is not null or salary_to is not null
                            ''', con=connection)
median = df_4_3['salary'].median()
mean = df_4_3['salary'].mean()
q_val = 0.95
quantile = df_4_3['salary'].quantile(q_val)
fig = px.scatter(df_4_3,
                 x=df_4_3.index,
                 y='salary',
                 title='salary',
                 labels={'x': 'index', 'y': 'salary'})
fig.add_hline(y=median, line_color="red", annotation_text="Median")
fig.add_hline(y=quantile, line_color="green", annotation_text=f"Quantile {q_val}")

print(f'Median {median}\n Mean {round(mean, 1)}\nQuantile {q_val} {quantile}')
fig.show()

Median 60000.0
 Mean 84314.9
Quantile 0.95 225000.0


In [16]:
df_4_3['salary'].value_counts()

salary
50000.0     1214
40000.0     1157
100000.0    1106
60000.0     1047
80000.0      894
            ... 
26995.0        1
50125.0        1
60040.0        1
40237.0        1
195275.0       1
Name: count, Length: 1423, dtype: int64

В основном вакансии находятся на территории РФ.

Примерно у половины вакансий не заполнено хотябы одно полу с зарплатой. Это означает, что если строить модель машинного обучения, которая будет учитывать зарплату, то половину данных прийдется откинуть, однако 24 тысячь вполне может хватить.

Неплохой средний уровень зарплаты. Здесь стоит провести более качественный анализ, учитывая, что высокооплачиваемые вакансии могут оттягивать среднее значение. Для более точной оценки стоит найти среднее по каждой профессии, и на основе этих данных посчитать среднее по всем професиям.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [36]:
query_5_1 = f'''
                select count(v.id), e.name
                from vacancies v
                join employers e on v.employer_id = e.id
                group by e.name
                order by 1 desc
                limit 5
'''

In [37]:
pd.read_sql_query(query_5_1, con=connection)

,count,name
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [61]:
query_5_2 = f'''
                select count(distinct e.id) as employers, a.name
                from areas a
                left join vacancies v on a.id = v.area_id
                left join employers e on a.id = e.area
                where v.area_id is null
                group by a.name
                order by 1 desc
'''

In [62]:
df_5_2 = pd.read_sql_query(query_5_2, con=connection)
df_5_2

,employers,name
0,410,Россия
1,207,Казахстан
2,75,Московская область
3,19,Краснодарский край
4,18,Беларусь
...,...,...
588,0,Кувандык
589,0,Куйбышев
590,0,Куйбышево (Ростовская область)
591,0,Култаево


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [25]:
query_5_3 = f'''
                select e.name, count(distinct a.id)
                from employers e
                left join vacancies v on e.id = v.employer_id
                left join areas a on v.area_id = a.id
                where v.id is not null
                group by e.name
                order by 2 desc
'''

In [26]:
pd.read_sql_query(query_5_3, con=connection)

,name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [33]:
query_5_4 = f'''
                select count(e.id)
                from employers e
                left join employers_industries ei on ei.employer_id = e.id
                left join industries i on i.id = ei.industry_id
                where i.id is null
'''

In [34]:
pd.read_sql_query(query_5_4, con=connection)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [39]:
query_5_5 = f'''
                select e.name
                from employers_industries ei
                left join employers e on ei.employer_id = e.id
                left join industries i on i.id = ei.industry_id
                group by e.name
                having count(distinct i.id) = 4
                order by 1
                limit 3
'''

In [40]:
pd.read_sql_query(query_5_5, con=connection)

,name
0,101 Интернет
1,21vek.by
2,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [43]:
query_5_6 = f'''
                select count(e.id)
                from employers e
                left join employers_industries ei on ei.employer_id = e.id
                left join industries i on i.id = ei.industry_id
                where i.name = 'Разработка программного обеспечения'
'''

In [44]:
pd.read_sql_query(query_5_6, con=connection)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8).

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API.

In [67]:
url = "https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8"

data = pd.read_html(url, header=0)
cities_df = data[0]
cities_df = cities_df['Город']

In [75]:
query_5_7 = f'''
                select a.name as cities, count(v.id)
                from vacancies v
                join employers e on e.id = v.employer_id
                join areas a on a.id = v.area_id
                where e.name = 'Яндекс'
                group by a.name
'''

In [77]:
df = pd.read_sql_query(query_5_7, con=connection)
df = df.merge(cities_df, left_on='cities', right_on='Город')
df = df.drop('Город', axis=1)
df.loc[df.shape[0]] = {'cities': 'Total', 'count': df['count'].sum()}
df

,cities,count
0,Волгоград,24
1,Воронеж,32
2,Екатеринбург,39
3,Казань,25
4,Краснодар,30
5,Красноярск,23
6,Москва,54
7,Нижний Новгород,36
8,Новосибирск,35
9,Омск,21


***

In [63]:
df_5_2['employers'].value_counts()

employers
0      407
1      120
2       20
3       11
4        8
5        6
6        5
7        2
9        2
10       2
16       2
18       2
207      1
15       1
17       1
19       1
75       1
410      1
Name: count, dtype: int64

In [64]:
mask = df_5_2['employers'] > 3

fig_5_2 = px.bar(
    df_5_2[mask],
    x='name',
    y='employers',
    text='name',
    title='Распределение работадателей по регионам, у которх нет вакансий')

fig_5_2.update_traces(texttemplate='%{text}', textposition='outside')
fig_5_2.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_5_2.show()

Большое количество IT-компаний, которые ищут сотрудников. И в основном они, по имеющимся данным, находятся в России

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [20]:
query_6_1 = f'''
                select count(id)
                from vacancies
                where name like '%data%' or
                      name like '%Data%' or
                      name like '%DATA%' or
                      name like '%Данн%' or
                      name like '%данн%' or
                      name like '%ДАНН%'
'''

In [21]:
pd.read_sql_query(query_6_1, con=connection)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста?
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.


In [28]:
query_6_2 = f'''
                select count(*)
                from vacancies
                where (lower(name) like '%data scientist%' or
                      lower(name) like '%data science%' or
                      lower(name) like '%исследователь данных%' or
                      (name like '%ML%' and name not like '%HTML%' and name not like '%QML%') or
                      lower(name) like '%machine learning%' or
                      lower(name) like '%машинн%обучен%') and
                      (lower(name) like '%junior%' or
                      experience = 'Нет опыта' or
                      employment = 'Стажировка')
'''

In [29]:
pd.read_sql_query(query_6_2, con=connection)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [32]:
query_6_3 = f'''
                select count(*)
                from vacancies
                where (lower(name) like '%data scientist%' or
                      lower(name) like '%data science%' or
                      lower(name) like '%исследователь данных%' or
                      (lower(name) like '%ml%' and name not like '%HTML%' and name not like '%QML%') or
                      lower(name) like '%machine learning%' or
                      lower(name) like '%машинн%обучен%') and
                      (lower(key_skills) like '%sql%' or
                      lower(key_skills) like '%postgres%')
'''

In [33]:
pd.read_sql_query(query_6_3, con=connection)

,count
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [4]:
query_6_4 = f'''
                select count(*)
                from vacancies
                where (name ilike '%data scientist%' or
                      name ilike '%data science%' or
                      name ilike '%исследователь данных%' or
                      (name ilike '%ml%' and name not like '%HTML%' and name not like '%QML%') or
                      name ilike '%machine learning%' or
                      name ilike '%машинн%обучен%') and
                      key_skills ilike '%python%'
'''

In [5]:
pd.read_sql_query(query_6_4, con=connection)

,count
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [16]:
query_6_5 = f'''
                select round(avg(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, chr(9), '')) + 1), 2) as cnt_skills
                from vacancies
                where name ilike '%data scientist%' or
                      name ilike '%data science%' or
                      name ilike '%исследователь данных%' or
                      (name like '%ML%' and name not like '%HTML%') or
                      name ilike '%machine learning%' or
                      name ilike '%машинн%обучен%'
'''

In [17]:
pd.read_sql_query(query_6_5, con=connection)

,cnt_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*).

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [ ]:
query_6_5 = f'''
                select experience, round(avg(coalesce((salary_to + salary_from) / 2, salary_to, salary_from)))
                from vacancies
                where name ilike '%data scientist%' or
                      name ilike '%data science%' or
                      name ilike '%исследователь данных%' or
                      (name like '%ML%' and name not like '%HTML%') or
                      name ilike '%machine learning%' or
                      name ilike '%машинн%обучен%'
                group by experience
'''

In [59]:
pd.read_sql_query(query_6_5, con=connection)

,experience,round
0,Более 6 лет,NaN
1,Нет опыта,74643.0
2,От 1 года до 3 лет,139675.0
3,От 3 до 6 лет,243115.0


***

На первый взгляд, вакансий связанных с Data Science не так много относительно всех данных представленных в таблице. Высокая средняя зарплата, относительно небольшое среднее число ключевых навыков, необжодимых для работы. Новичкам сложнее найти работу в сфере DS.

# Общий вывод по проекту

При дальнйшем анализе стоит учитывать, что в регионах представлены как страны так и субъекты стран.

Если мы планируем построить модель, отображающую зависимости факторов вакансий, то объем данных, на первый взгляд позволяет это сделать. Однако, зависимости в таких сферах как DS отобразятся лучьше (полагем, что количество вакансий в области DS близко к доминирующей позиции). Чтобы исправить это нам нужны другие источники данных в добавок.

In [17]:
query_ = f'''
                select count(*)
                from vacancies
                where lower(name) like '%junior%' or
                      experience = 'Нет опыта' or
                      employment = 'Стажировка'
'''
pd.read_sql_query(query_, con=connection)

,count
0,7933


Количество вакансий для новичко составляет, примерно 1/5, что не плохо для рынка труда.

Если мы собираемся построить модель для предсказания зарплаты, то нужно очистить данные, провести разведовательный анализ.

В настоящее время быстро развивается IT-сегмент. Возможно сейчас это не самая популярныя часть рынка труда, но помере индустриального развития, данный сегмент может стать доминирующим. Кампании стоит обратить на него внимание.